In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, isnull, sum, median, mean, first, max, min
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()

25/01/29 17:12:29 WARN Utils: Your hostname, Vasileioss-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.0.67 instead (on interface en0)
25/01/29 17:12:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/29 17:12:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load data

In [3]:
df=spark.read.csv('../data/cust_data.csv',inferSchema=True,header=True)

In [4]:
df.show()

+---+------+---+-------------------+---------+------------------------------+---------+--------+------------------+-------------------+-------+-----+
| id|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|   Tenure|App_User|Num_website_visits|Acquisition_Channel|Revenue|Label|
+---+------+---+-------------------+---------+------------------------------+---------+--------+------------------+-------------------+-------+-----+
|  1|  Male| 44|                  1|       28|                             0|> 2 Years|     Yes|              NULL|                 26|  40.45|    1|
|  2|  Male| 76|                  1|        3|                             0| 1-2 Year|      No|              NULL|                 26|  33.54|    0|
|  3|  Male| 47|                  1|       28|                             0|> 2 Years|     Yes|              NULL|                 26|  38.29|    1|
|  4|  Male| 21|                  1|       11|                             1| < 1 Year|      No|    

In [5]:
print(f"Number of rows: {df.count()} and columns: {len(df.columns)}")

Number of rows: 381109 and columns: 12


In [6]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Has_Mobile_Contract: integer (nullable = true)
 |-- Area_Code: integer (nullable = true)
 |-- Currently_Holds_Second_Product: integer (nullable = true)
 |-- Tenure: string (nullable = true)
 |-- App_User: string (nullable = true)
 |-- Num_website_visits: string (nullable = true)
 |-- Acquisition_Channel: integer (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Label: integer (nullable = true)



In [7]:
df.describe().show()

25/01/29 17:12:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------+-----------------+--------------------+------------------+------------------------------+---------+--------+------------------+-------------------+-----------------+-------------------+
|summary|                id|Gender|              Age| Has_Mobile_Contract|         Area_Code|Currently_Holds_Second_Product|   Tenure|App_User|Num_website_visits|Acquisition_Channel|          Revenue|              Label|
+-------+------------------+------+-----------------+--------------------+------------------+------------------------------+---------+--------+------------------+-------------------+-----------------+-------------------+
|  count|            381109|381109|           381109|              381101|            381108|                        381109|   381108|  381109|                 0|             381109|           381109|             381109|
|   mean|          190555.0|  NULL|38.82248385632457|   0.997869331227155| 26.38884778068159|            0.458210118

# Count missing values

In [8]:
null_counts = [sum(when(isnull(col), 1).otherwise(0)).alias(col) for col in df.columns]
null_df = df.select(null_counts)

result = null_df.toPandas().transpose().reset_index()
result['null_percentage'] = (result[0] / df.count() * 100).round(3)

schema = StructType([
    StructField('column', StringType(), False),
    StructField('null_counts', LongType(), False),
    StructField('null_percentage', DoubleType(), False)
])

result_df = df.sparkSession.createDataFrame(
    result.rename(
        columns={
            "index": "column_name", 
            0: "null_count"
        }
    ),
    schema=schema
)

result_df.orderBy(col('null_counts').desc()).show()

+--------------------+-----------+---------------+
|              column|null_counts|null_percentage|
+--------------------+-----------+---------------+
|  Num_website_visits|     381109|          100.0|
| Has_Mobile_Contract|          8|          0.002|
|              Tenure|          1|            0.0|
|           Area_Code|          1|            0.0|
|            App_User|          0|            0.0|
| Acquisition_Channel|          0|            0.0|
|              Gender|          0|            0.0|
|                  id|          0|            0.0|
|                 Age|          0|            0.0|
|             Revenue|          0|            0.0|
|Currently_Holds_S...|          0|            0.0|
|               Label|          0|            0.0|
+--------------------+-----------+---------------+



# Observations
- I think `Num_website_visits` I can drop from the dataset because it is impossible to fill in or impute. 
- The features `Has_Mobile_Contract`, `Area_Code`, `Tenure` have very few missing values so I can try to impute them
- I can also create an **indicator of missing values** in the dataset. That might give more context to the model as to why there are missing values.
- The `Age` column has some out of range values such as -1 and 190. I will need to investigate and deal with that.
- The `Revenue` column seems to have some outliers. Need to investigate that too.

# Continue with the exploration

In [9]:
df = df.drop('id', 'Num_website_visits')

In [10]:
df.filter(df.Age == -1).show()

+------+---+-------------------+---------+------------------------------+---------+--------+-------------------+-------+-----+
|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|   Tenure|App_User|Acquisition_Channel|Revenue|Label|
+------+---+-------------------+---------+------------------------------+---------+--------+-------------------+-------+-----+
|  Male| -1|                  1|       18|                             1| 1-2 Year|      No|                 26|   2.63|    0|
|  Male| -1|                  1|        8|                             0|> 2 Years|     Yes|                124|  22.26|    1|
|  Male| -1|                  1|        8|                             0| 1-2 Year|     Yes|                 26|  30.58|    1|
|Female| -1|                  1|       31|                             1| < 1 Year|      No|                152|   2.63|    0|
+------+---+-------------------+---------+------------------------------+---------+--------+-------------------

In [11]:
df.filter(df['Age'] > 85).show()

+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+
|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|  Tenure|App_User|Acquisition_Channel|Revenue|Label|
+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+
|  Male|190|                  1|        6|                             1|< 1 Year|      No|                152|  19.14|    0|
+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+



These out of range values at the Age column might indicate a problem with the system or just a data quality issue. I don't think I should remove them from the dataset but there is no clear way how to substitue them. I think I might just impute them using the mean.

In [12]:
df.select('Revenue').describe().show()

+-------+-----------------+
|summary|          Revenue|
+-------+-----------------+
|  count|           381109|
|   mean|30.56480615780831|
| stddev|17.21329544913057|
|    min|             2.63|
|    max|           540.17|
+-------+-----------------+



In [13]:
df.filter(df['Revenue'] == df.select(max('Revenue')).first()[0]).show()

+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+
|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|  Tenure|App_User|Acquisition_Channel|Revenue|Label|
+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+
|  Male| 26|                  1|       28|                             0|< 1 Year|     Yes|                156| 540.17|    1|
|Female| 53|                  1|       28|                             1|1-2 Year|      No|                 26| 540.17|    0|
|  Male| 47|                  1|       28|                             0|1-2 Year|     Yes|                 42| 540.17|    0|
|  Male| 46|                  1|       28|                             0|1-2 Year|     Yes|                124| 540.17|    0|
+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------

In [14]:
df.filter(df['Revenue'] == df.select(min('Revenue')).first()[0]).show()

+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+
|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|  Tenure|App_User|Acquisition_Channel|Revenue|Label|
+------+---+-------------------+---------+------------------------------+--------+--------+-------------------+-------+-----+
|Female| 24|                  1|       33|                             0|< 1 Year|     Yes|                160|   2.63|    0|
|  Male| 37|                  1|        6|                             0|1-2 Year|     Yes|                156|   2.63|    1|
|Female| 26|                  1|        8|                             0|< 1 Year|      No|                160|   2.63|    0|
|Female| 38|                  1|       35|                             1|1-2 Year|      No|                152|   2.63|    0|
|Female| 62|                  1|       48|                             0|1-2 Year|     Yes|                 15|   2.63

In [15]:
df.filter(df['Revenue'] == df.select(min('Revenue')).first()[0]).count()

64877

In [16]:
df.groupBy('Label').count().show()

+-----+------+
|Label| count|
+-----+------+
|    1| 46710|
|    0|334399|
+-----+------+



The target value is massively imballanced

In [17]:
df.groupBy('Gender').count().show()

+------+------+
|Gender| count|
+------+------+
|Female|175020|
|  Male|206089|
+------+------+



In [18]:
df.groupby('Has_Mobile_Contract').count().show()

+-------------------+------+
|Has_Mobile_Contract| count|
+-------------------+------+
|               NULL|     8|
|                  1|380289|
|                  0|   812|
+-------------------+------+



In [19]:
df.filter(df['Has_Mobile_Contract'] == 1).groupby('Label').count().show()

+-----+------+
|Label| count|
+-----+------+
|    1| 46666|
|    0|333623|
+-----+------+



In [20]:
df.filter(df['Currently_Holds_Second_Product'] == 1).groupby('Label').count().show()

+-----+------+
|Label| count|
+-----+------+
|    1|   158|
|    0|174470|
+-----+------+



In [21]:
df.filter(df['Currently_Holds_Second_Product'] == 1).groupby('Gender').count().show()

+------+-----+
|Gender|count|
+------+-----+
|Female|87949|
|  Male|86679|
+------+-----+



In [22]:
df.groupBy('Label').sum('Revenue').show()

+-----+--------------------+
|Label|        sum(Revenue)|
+-----+--------------------+
|    1|  1476246.8900000276|
|    0|1.0172275819997048E7|
+-----+--------------------+



In [27]:
df.select('Acquisition_Channel').distinct().count()

155

In [28]:
df.select('Area_Code').distinct().count()

54

In [30]:
df.groupBy('Tenure').count().show()

+---------+------+
|   Tenure| count|
+---------+------+
|     NULL|     1|
|> 2 Years| 16007|
| < 1 Year|164786|
| 1-2 Year|200315|
+---------+------+

